In [ ]:
import pandas as pd
import dapla as dp
import qgrid
import numpy as np

## Relativt elegant, men kompleks datainnlast

In [ ]:
#dp.show('/felles/mock_sysselsatte')

In [ ]:
# Man kan skrive inn understreker i tall, for bedre lesbarhet, men de vil senere bli ignorert.
pop_size = 10_000
# Siste årstall, skal være siste årstall+1
aar_serie = (2019, 2024)

In [ ]:
# List-comprehensions for å generere alle paths, med tuple unpacking for årstall
pop_in_paths = [f'/felles/mock_sysselsatte/population_{i}_{pop_size}' for i in range( *aar_serie )]
comp_in_paths = [f'/felles/mock_sysselsatte/companies_{i}_{pop_size}' for i in range( *aar_serie )]

In [ ]:
# Dictionary som dataframene skal stappes inn i
dfs = {}
# Loop gjennom alle pathene vi lagde med comprehensionene
for in_path in pop_in_paths + comp_in_paths:
    print('Laster inn', in_path)
    # Lag keys fra pathene, og les inn dataframes fra daplalagringen på de dem
    dfs['_'.join(in_path.split('/')[-1].split('_')[:2])] = dp.read_pandas(in_path)

In [ ]:
# Eksempler på henting av data ut fra dictionaryen
comp2020 = dfs['companies_2020'].drop('employee_points', axis = 'columns')
comp2020.tail(5)

In [ ]:
pop2020 = dfs['population_2020']
pop2020

## Koble (merge) data fra selskaper på personer

In [ ]:
popcomp2020 = pop2020.merge(comp2020, 
                            how = 'left', 
                            on = 'work_id' )
popcomp2020

In [ ]:
# Personer registrert med arbeidssted
popcomp2020[~popcomp2020['work_id'].isnull()].head()

## Start Qgrid

In [ ]:
qgrid_widget = qgrid.show_grid(popcomp2020, show_toolbar = True)
qgrid_widget

In [ ]:
df_selection = qgrid_widget.get_changed_df()
df_selection

In [ ]:
# After upgrading to 1.0.3
def handle_json_updated(event, qgrid_widget):
    # exclude 'viewport_changed' events since that doesn't change the DataFrame
    if (event['triggered_by'] != 'viewport_changed'):
        print(qgrid_widget.get_changed_df())

qgrid_widget.on('json_updated', handle_json_updated)

In [ ]:
df_selection = qgrid_widget.get_changed_df()
row_ids = df_selection.index.tolist()

### Her bør du logge endringene du gjør ett eller annet sted...

In [ ]:
# Dette er sånt som hadde vært smart å logget, før man påførte det på datasettet
diff = popcomp2020.loc[row_ids].compare(df_selection)
diff

In [ ]:
# Navn på datasett endret, brukernavn, tidspunkt, endringskommentar

### Du kan overskrive dataene i det originale datasettet, men hva gjør du da for å endre det tilbake?

In [ ]:
# Skriv over rader i orginaldataframe med endringene
#for i, row in df_selection.iterrows():
#    popcomp2020.loc[i] = row

In [ ]:
#popcomp2020.loc[row_ids]